In [38]:
import pickle
import seaborn as sns
sns.set()
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import glob
from scipy.signal import savgol_filter
import imageio
import pickle
import glob
import cv2
from moviepy.editor import *

In [100]:
base_path = "log/2000/data/vary_1.4_0.8_/"

In [101]:
filename = glob.glob(base_path+"*.mp4")[0]
vid = imageio.get_reader(filename, 'ffmpeg')
images = []
for i, im in enumerate(vid):
    images += [im]
images = np.array(images)

In [102]:
print images.shape

(10000, 500, 500, 3)


In [103]:
class Prep_Data():
    def __init__(self, path=None, filter_size=401):
        with open(path, "rb") as p:
            data = pickle.load(p)
        
        self.vels = data["vels"]
        self.kds = data["kds"]
        self.dvs = data["dvs"]        
        self.states = data["states"]
        self.vels_smooth = savgol_filter(self.vels, filter_size, 3)
        self.label = path.split("data/")[-1].split("_/")[0]

In [104]:
data = Prep_Data(path=base_path+"vary_1.4_0.8_.pkl")
dvels = data.dvs
rknee = savgol_filter(data.states.T[4], 401, 3)
lknee = savgol_filter(data.states.T[6], 401, 3)

In [105]:
def make_vel_plot(vels, dvels):
    sns.set_style("darkgrid")
    sns.set_context("paper", font_scale=2)
    blues = sns.dark_palette("blue", 8)
    
    f, ax = plt.subplots(figsize=(10,5))
    times = np.arange(vels.shape[0])*0.002
    ax.plot(np.arange(dvels.shape[0])*0.002, dvels, "--", color=blues[3], linewidth=4, label="desired velocity")
    ax.plot(times, vels, "-", color=blues[7], linewidth=1, label="hip velocity")
    if vels.shape[0] > 2:
        ax.plot(times[vels.shape[0]-1], vels[-1], "o", color=blues[7], linewidth=6, label="current hip velocity")
    else:
        ax.plot(0., 0., "o", color=blues[7], linewidth=6, label="current hip velocity")
    ax.set_ylim(0., 1.5)
    ax.set_xlim(0, 21)
    ax.legend(loc=4)
    
    f.text(0.5, 0.05, 'time(s)', ha='center')
    f.text(0.05, 0.5, 'hip velocity (m/s)', va='center', rotation='vertical')
    
    return f

In [106]:
video_path = base_path + "frames/"
if not os.path.exists(video_path):
    os.makedirs(video_path)
for i in range(10000):
    if i % 25 == 0:
        fig = make_vel_plot(data.vels_smooth[0:i], dvels)
        plt.savefig(video_path+str("%05d" % i)+".jpg")
        plt.clf()
        img = imageio.imread(video_path+str("%05d" % i)+".jpg")
        img = cv2.resize(img, (500, 250))
        frame = np.concatenate([images[i], img], axis=0)
        imageio.imwrite(video_path+str("%05d" % i)+".jpg", frame)
    
#     vel_images += []
# vel_images = np.array(vel_images)
# print vel_images.shape[0]

In [107]:
video_path = base_path + "frames/"
if not os.path.exists(video_path):
    os.makedirs(video_path)
frames = []
for i in range(10000):
    if i % 25 == 0:
        frames += [imageio.imread(video_path+str("%05d" % i)+".jpg")]
frames = np.array(frames)
print frames.shape


(400, 750, 500, 3)


In [108]:
frames = list(frames)

In [109]:
clip = ImageSequenceClip(frames, fps=20)
clip.write_videofile(base_path+base_path.split("/")[3]+"demo.mp4", audio=False)

[MoviePy] >>>> Building video log/2000/data/vary_1.4_0.8_/vary_1.4_0.8_demo.mp4
[MoviePy] Writing video log/2000/data/vary_1.4_0.8_/vary_1.4_0.8_demo.mp4


100%|██████████| 401/401 [00:01<00:00, 314.39it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: log/2000/data/vary_1.4_0.8_/vary_1.4_0.8_demo.mp4 

